In [1]:
import pandas as pd

In [2]:
fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.gff"
df = pd.read_csv(fn, sep='\t', header=None)
df.columns = ['chr', 'src', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df.head()

,chr,src,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11121.0,24894.0,.,+,.,ID=ENSG00000290825.2;gene_id=ENSG00000290825.2...
1,chr1,HAVANA,transcript,11426.0,14409.0,.,+,.,ID=ENST00000832828.1;Parent=ENSG00000290825.2;...
2,chr1,HAVANA,exon,11426.0,11671.0,.,+,.,ID=exon:ENST00000832828.1:1;Parent=ENST0000083...
3,chr1,HAVANA,exon,12010.0,12227.0,.,+,.,ID=exon:ENST00000832828.1:2;Parent=ENST0000083...
4,chr1,HAVANA,exon,12613.0,12721.0,.,+,.,ID=exon:ENST00000832828.1:3;Parent=ENST0000083...


In [10]:
def get_ttype(s):
    temp = s.split(';')
    ttype = None
    tid = None
    pid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            tid = kv[1]
        elif kv[0] == 'Parent':
            pid = kv[1]
        elif kv[0] == 'transcript_type':
            ttype = kv[1]
            break
    return tid, pid, ttype

def get_gtype(s):
    temp = s.split(';')
    gtype = None
    gid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            gid = kv[1]
        elif kv[0] == 'gene_type':
            gtype = kv[1]
            break
    return gid, gtype

def get_parent(s):
    temp = s.split(';')
    gtype = None
    gid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'Parent':
            pid = kv[1]
            break
    return pid

In [11]:
pc_genes = []
for i, row in df.iterrows():
    if row['feature'] == 'gene':
        gid, gtype = get_gtype(row['attribute'])
        if gtype == "protein_coding" or gtype == 'TR_C_gene': # based on obs
            pc_genes.append(gid)


pc_g2t = dict()
for i, row in df.iterrows():
    if row['feature'] == 'transcript':
        tid, pid, ttype = get_ttype(row['attribute'])
        if ttype == "protein_coding" or ttype == "TR_C_gene":
            assert pid in pc_genes
            if pid not in pc_g2t:
                pc_g2t[pid] = [tid]
            else:
                pc_g2t[pid].append(tid)

In [13]:
pc_tids = []
for x in pc_g2t:
    for y in pc_g2t[x]:
        pc_tids.append(y)
print(len(pc_tids)) # checks out

64674


In [16]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.pc_only.gff"
with open(out_fn, 'w') as out_fh:
    for i, row in df.iterrows():
        do_print = False
        if row['feature'] == 'gene':
            gid, gtype = get_gtype(row['attribute'])
            if gid in pc_genes:
                do_print = True
        elif row['feature'] == 'transcript':
            tid, pid, ttype = get_ttype(row['attribute'])
            if tid in pc_tids:
                do_print = True
        elif row['feature'] == 'exon':
            pid = get_parent(row['attribute'])
            if pid in pc_tids:
                do_print = True
        if do_print:
            s = '\t'.join(str(x) for x in row)
            out_fh.write(s + '\n')